In [0]:
%run ../source_to_bronze/utils


In [0]:
spark.sql("USE CATALOG assignment;")
spark.sql("USE SCHEMA silver;")


In [0]:
bronze_processed_path = "/Volumes/assignment/bronze/raw_data/processed"

employee_bronze_df = spark.read.option("header", True).csv(f"{bronze_processed_path}/employee")
department_bronze_df = spark.read.option("header", True).csv(f"{bronze_processed_path}/department")
country_bronze_df = spark.read.option("header", True).csv(f"{bronze_processed_path}/country")

display(employee_bronze_df)


In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

employee_schema = StructType([
    StructField("EmployeeID", StringType(), True),
    StructField("EmployeeName", StringType(), True),
    StructField("Department", StringType(), True),
    StructField("Country", StringType(), True),
    StructField("Salary", IntegerType(), True),
    StructField("Age", IntegerType(), True)
])


In [0]:
employee_silver_df = spark.read.schema(employee_schema).option("header", True).csv(f"{bronze_processed_path}/employee")


In [0]:
department_schema = StructType([
    StructField("DepartmentID", StringType(), True),
    StructField("DepartmentName", StringType(), True)
])

country_schema = StructType([
    StructField("CountryCode", StringType(), True),
    StructField("CountryName", StringType(), True)
])

department_silver_df = spark.read.schema(department_schema).csv(f"{bronze_processed_path}/department")
country_silver_df = spark.read.schema(country_schema).csv(f"{bronze_processed_path}/country")


In [0]:
employee_silver_df = convert_columns_to_snake_case(employee_silver_df)
department_silver_df = convert_columns_to_snake_case(department_silver_df)
country_silver_df = convert_columns_to_snake_case(country_silver_df)


In [0]:
employee_silver_df = add_load_date_column(employee_silver_df)
department_silver_df = add_load_date_column(department_silver_df)
country_silver_df = add_load_date_column(country_silver_df)


In [0]:
dbutils.fs.mkdirs("/Volumes/assignment/silver/silverdata/silver")



In [0]:
silver_path = "/Volumes/assignment/silver/silverdata/silver"


In [0]:
employee_silver_df.write.format("delta").mode("overwrite").save(f"{silver_path}/dim_employee")
department_silver_df.write.format("delta").mode("overwrite").save(f"{silver_path}/dim_department")
country_silver_df.write.format("delta").mode("overwrite").save(f"{silver_path}/dim_country")


In [0]:
display(dbutils.fs.ls("/Volumes/assignment/bronze/raw_data/silver"))


In [0]:
display(dbutils.fs.ls("/Volumes/assignment/bronze/raw_data/silver"))
